---
## 종목코드 crawling
### (참조 : http://excelsior-cjh.tistory.com/109?category=975542 [EXCELSIOR])
---

In [151]:
import pandas as pd
import numpy as np
from time import sleep


# 한국거래소에 등록된 종목코드 crawling
code_df = pd.read_html('http://kind.krx.co.kr/corpgeneral/corpList.do?method=download&searchType=13', header=0)[0]

# 18.5.26기준 총 2,193개 종목
print(code_df.shape)
print(code_df.columns)
print(code_df.iloc[:10,])

# pandas columns을 한글에서 영어로 바꿈
code_df.columns = ['comp_nm', 'code', 'line', 'main_prod', 'listed_dt', 'closing_ym', 'owner', 'homepage', 'region']

(2193, 9)
Index(['회사명', '종목코드', '업종', '주요제품', '상장일', '결산월', '대표자명', '홈페이지', '지역'], dtype='object')
        회사명    종목코드                         업종  \
0       BYC    1460                   봉제의복 제조업   
1    CJ CGV   79160  영화, 비디오물, 방송프로그램 제작 및 배급업   
2    DB손해보험    5830                        보험업   
3   DGB금융지주  139130                     기타 금융업   
4     DSR제강   69730                  1차 철강 제조업   
5      KB금융  105560                     기타 금융업   
6   KC그린홀딩스    9440                     기타 금융업   
7  KISCO홀딩스    1940                     기타 금융업   
8   LS전선아시아  229640                     기타 금융업   
9     MH에탄올   23150                  알코올음료 제조업   

                                             주요제품         상장일  결산월  \
0                메리야스,란제리 제조,도매/건축공사/부동산 임대,분양,공급  1975-06-02  12월   
1                                     영화상영,영화관 운영  2004-12-24  12월   
2                       자동차보험,화재보험,해상보험,특종보험,장기보험  1973-06-28  12월   
3                                            지주회사  2011-06-07  12월   


In [152]:
# 종목코드 6자리로 처리
code_df.code = code_df.code.map('{:06d}'.format)

# 결산월 / 대표자명 / 홈페이지 / 지역 제외
code_df = code_df[code_df.columns[:5]]

In [153]:
# 업종(line) 및 주요제품(main_prod)를 list 형태로 변환하여 입력
line_list = {}
main_prod_list = {}

for i in range(len(code_df)):  
    ### 종목명 ### 
    tmp_line_list = code_df.iloc[i,2]
    if not isinstance(tmp_line_list,float):
        tmp_line_list = tmp_line_list.replace(" ","").replace("/",",").split(',') 
        code_df.iloc[i,2] = tmp_line_list # list형태로 변환
        
        # 종목명 dict 생성 
        for line_nm in tmp_line_list:
            if line_nm not in line_list:
                line_list[line_nm] = 1
            else:                
                line_list[line_nm] = line_list[line_nm] + 1
        
    ### 주요제품 ###    
    tmp_main_prod_list = code_df.iloc[i,3]
    if not isinstance(tmp_main_prod_list,float): # nan처리
        tmp_main_prod_list = tmp_main_prod_list.replace(" ","").replace("/",",").split(',')    
        code_df.iloc[i,3] = tmp_main_prod_list
            
        # 주요제품 dict 생성   
        for main_prod_nm in tmp_main_prod_list:
            if main_prod_nm not in main_prod_list:
                main_prod_list[main_prod_nm] = 1
            else:                
                main_prod_list[main_prod_nm] = main_prod_list[main_prod_nm] + 1

In [154]:
code_df.head()

,comp_nm,code,line,main_prod,listed_dt
0,BYC,001460,[봉제의복제조업],"[메리야스, 란제리제조, 도매, 건축공사, 부동산임대, 분양, 공급]",1975-06-02
1,CJ CGV,079160,"[영화, 비디오물, 방송프로그램제작및배급업]","[영화상영, 영화관운영]",2004-12-24
2,DB손해보험,005830,[보험업],"[자동차보험, 화재보험, 해상보험, 특종보험, 장기보험]",1973-06-28
3,DGB금융지주,139130,[기타금융업],[지주회사],2011-06-07
4,DSR제강,069730,[1차철강제조업],"[와이어로프, 각종경강선, 철선제품, PC강선, 아연도강연선제조]",2003-01-28


In [155]:
k = 10
top_k_line_list = sorted(line_list, key=line_list.get, reverse=True)[:k]
top_k_main_prod_list = sorted(main_prod_list, key=main_prod_list.get, reverse=True)[:k]

top_k_line_list = [(line_nm, line_list[line_nm]) for line_nm in top_k_line_list]
top_k_main_prod_list = [(main_prod_nm, main_prod_list[main_prod_nm]) for main_prod_nm in top_k_main_prod_list]


# 주요 업종 및 주요상품 (Top 10개)
print(top_k_line_list)
print(top_k_main_prod_list)

[('소프트웨어개발및공급업', 118), ('특수목적용기계제조업', 117), ('전자부품제조업', 116), ('기타금융업', 112), ('자동차신품부품제조업', 91), ('의약품제조업', 78), ('기타화학제품제조업', 76), ('통신및방송장비제조업', 73), ('금융지원서비스업', 60), ('1차철강제조업', 56)]
[('판매', 117), ('도매', 110), ('부동산임대', 57), ('기업인수합병', 38), ('수출입', 30), ('무역', 27), ('건축공사', 25), ('도소매', 23), ('토목공사', 22), ('지주회사', 22)]


---
## 종목별 일별 시세 crawling
---

In [189]:
# 일별시세 url
code = '000660'

ind = 1
daily_res = pd.DataFrame()
print("종목 '{}' 일별시세 crawling 중".format(str(code_df.loc[code_df.code == code].comp_nm.values[0])))

while True:    
    print("page # {} is crawling".format(str(ind)))
    url = "http://finance.naver.com/item/sise_day.nhn?code={}&page={}".format(str(code), str(ind))
    daily_res = daily_res.append(pd.read_html(url, header=0)[0], ignore_index=True)
    tmp = requests.get(url)                   
    tmp_b = BeautifulSoup(tmp.text, 'lxml') 
#     if ind == 2:
    if int(tmp_b.find('table', {'class':'Nnavi'}).find('td', {'class':'on'}).text.replace("\n","")) != ind:
        break
    ind += 1
    sleep(0.05)

# nan이 포함된 row 제거
daily_res = daily_res[np.isfinite(daily_res['종가'])]
daily_res[daily_res.columns[1:]] = daily_res[daily_res.columns[1:]].astype(int)
daily_res.head()

종목 'SK하이닉스' 일별시세 crawling 중
page # 1 is crawling
page # 2 is crawling
page # 3 is crawling
page # 4 is crawling
page # 5 is crawling
page # 6 is crawling
page # 7 is crawling
page # 8 is crawling
page # 9 is crawling
page # 10 is crawling
page # 11 is crawling
page # 12 is crawling
page # 13 is crawling
page # 14 is crawling
page # 15 is crawling
page # 16 is crawling
page # 17 is crawling
page # 18 is crawling
page # 19 is crawling
page # 20 is crawling
page # 21 is crawling
page # 22 is crawling
page # 23 is crawling
page # 24 is crawling
page # 25 is crawling
page # 26 is crawling
page # 27 is crawling
page # 28 is crawling
page # 29 is crawling
page # 30 is crawling
page # 31 is crawling
page # 32 is crawling
page # 33 is crawling
page # 34 is crawling
page # 35 is crawling
page # 36 is crawling
page # 37 is crawling
page # 38 is crawling
page # 39 is crawling
page # 40 is crawling
page # 41 is crawling
page # 42 is crawling
page # 43 is crawling
page # 44 is crawling
page # 45 is 

page # 361 is crawling
page # 362 is crawling
page # 363 is crawling
page # 364 is crawling
page # 365 is crawling
page # 366 is crawling
page # 367 is crawling
page # 368 is crawling
page # 369 is crawling
page # 370 is crawling
page # 371 is crawling
page # 372 is crawling
page # 373 is crawling
page # 374 is crawling
page # 375 is crawling
page # 376 is crawling
page # 377 is crawling
page # 378 is crawling
page # 379 is crawling
page # 380 is crawling
page # 381 is crawling
page # 382 is crawling
page # 383 is crawling
page # 384 is crawling
page # 385 is crawling
page # 386 is crawling
page # 387 is crawling
page # 388 is crawling
page # 389 is crawling
page # 390 is crawling
page # 391 is crawling
page # 392 is crawling
page # 393 is crawling
page # 394 is crawling
page # 395 is crawling
page # 396 is crawling
page # 397 is crawling
page # 398 is crawling
page # 399 is crawling
page # 400 is crawling
page # 401 is crawling
page # 402 is crawling
page # 403 is crawling
page # 404 

,날짜,종가,전일비,시가,고가,저가,거래량
0,2018.05.25,95200,600,94300,97700,93900,4140840
1,2018.05.24,94600,700,95300,95900,94600,3220029
2,2018.05.23,95300,6200,91900,95500,91500,10378915
3,2018.05.21,89100,1200,88400,89400,87400,3262983
4,2018.05.18,87900,1200,87200,88000,85400,2187789


In [190]:
daily_res[:100]

,날짜,종가,전일비,시가,고가,저가,거래량
0,2018.05.25,95200,600,94300,97700,93900,4140840
1,2018.05.24,94600,700,95300,95900,94600,3220029
2,2018.05.23,95300,6200,91900,95500,91500,10378915
3,2018.05.21,89100,1200,88400,89400,87400,3262983
4,2018.05.18,87900,1200,87200,88000,85400,2187789
5,2018.05.17,86700,300,88400,88700,86000,2864278
6,2018.05.16,87000,2400,85000,87000,84800,3040180
7,2018.05.15,84600,800,86000,86000,83900,2239370
8,2018.05.14,85400,700,86100,86300,84500,1959236
9,2018.05.11,86100,300,86600,86800,85400,2107047
